# Notebook 8: Agentic Workflow Patterns

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 8 of 13  
**Level:** Advanced  
**Duration:** ~75 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand the core agentic workflow patterns
2. Implement the **Router Pattern** for task delegation
3. Build **Orchestrator-Workers** architectures
4. Apply the **Evaluator-Optimizer** loop for iterative refinement
5. Combine patterns for complex multi-agent systems
6. Integrate MCP servers into agentic workflows

## 1. The Agentic AI Paradigm Shift

### From Chat to Agency

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    THE EVOLUTION OF AI SYSTEMS                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   TRADITIONAL CHATBOT              AGENTIC AI SYSTEM                    │
│   ═══════════════════              ══════════════════                   │
│                                                                          │
│   User ──► LLM ──► Response        User ──► Agent ──┬──► Tool A        │
│                                              │       ├──► Tool B        │
│   • Single turn                              │       ├──► Database      │
│   • No actions                               │       └──► API           │
│   • Text only                                │                          │
│                                              ▼                          │
│                                         Reasoning                       │
│                                              │                          │
│                                              ▼                          │
│                                    More tools if needed                 │
│                                              │                          │
│                                              ▼                          │
│                                         Response                        │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   KEY CHARACTERISTICS OF AGENTIC SYSTEMS:                               │
│                                                                          │
│   🎯 Goal-Oriented    - Works toward objectives, not just responses     │
│   🔄 Iterative        - Multiple reasoning/action cycles                │
│   🛠️ Tool-Using       - Executes real actions via tools                 │
│   🧠 Autonomous       - Makes decisions without constant guidance       │
│   📊 Context-Aware    - Maintains state across interactions             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### MCP's Role in Agentic Systems

MCP provides the **standardized interface** between agents and their tools:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP IN AGENTIC ARCHITECTURES                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                         ┌─────────────┐                                 │
│                         │   AGENT     │                                 │
│                         │   (LLM)     │                                 │
│                         └──────┬──────┘                                 │
│                                │                                        │
│                         MCP Protocol                                    │
│                                │                                        │
│          ┌─────────────────────┼─────────────────────┐                  │
│          │                     │                     │                  │
│          ▼                     ▼                     ▼                  │
│   ┌─────────────┐       ┌─────────────┐       ┌─────────────┐          │
│   │ MCP Server  │       │ MCP Server  │       │ MCP Server  │          │
│   │ (Database)  │       │ (APIs)      │       │ (Files)     │          │
│   └─────────────┘       └─────────────┘       └─────────────┘          │
│                                                                          │
│   BENEFITS:                                                             │
│   • Uniform tool interface (JSON-RPC)                                   │
│   • Discoverable capabilities (tools/list)                              │
│   • Type-safe interactions (JSON Schema)                                │
│   • Pluggable architecture (swap servers easily)                        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. The Five Core Agentic Patterns

Based on industry research and real-world deployments:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    FIVE AGENTIC WORKFLOW PATTERNS                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   1. ROUTER                    2. PARALLELIZATION                       │
│   ══════════                   ═══════════════════                      │
│   Route to specialized         Execute multiple                         │
│   handlers based on            tasks simultaneously                     │
│   input classification                                                  │
│                                                                          │
│        ┌──► Handler A               ┌──► Task A ──┐                     │
│   In ──┼──► Handler B          In ──┼──► Task B ──┼──► Aggregate        │
│        └──► Handler C               └──► Task C ──┘                     │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   3. ORCHESTRATOR-WORKERS      4. EVALUATOR-OPTIMIZER                   │
│   ═══════════════════════      ══════════════════════                   │
│   Central coordinator           Iterative refinement                    │
│   delegates to workers          with feedback loop                      │
│                                                                          │
│        ┌─────────────┐              ┌──────────┐                        │
│        │ Orchestrator│              │ Generate │                        │
│        └──────┬──────┘              └────┬─────┘                        │
│          ┌────┼────┐                     │                              │
│          ▼    ▼    ▼                     ▼                              │
│        [W1] [W2] [W3]              ┌──────────┐                         │
│          └────┼────┘         ┌────►│ Evaluate │                         │
│               ▼              │     └────┬─────┘                         │
│          Synthesize          │          │ Pass/Fail                     │
│                              │          ▼                               │
│                              └──── Optimize ◄── (if fail)               │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   5. AUTONOMOUS AGENT                                                   │
│   ═══════════════════                                                   │
│   Self-directed with                                                    │
│   planning & memory                                                     │
│                                                                          │
│        ┌──────────────────────────────┐                                 │
│        │         AGENT LOOP           │                                 │
│        │  Plan → Act → Observe → Plan │                                 │
│        └──────────────────────────────┘                                 │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 3. Pattern 1: Router

### Concept

The **Router Pattern** classifies incoming requests and routes them to specialized handlers:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         ROUTER PATTERN                                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                        ┌─────────────────┐                              │
│   User Request ───────►│     ROUTER      │                              │
│                        │   (Classifier)  │                              │
│                        └────────┬────────┘                              │
│                                 │                                       │
│              ┌──────────────────┼──────────────────┐                    │
│              │                  │                  │                    │
│              ▼                  ▼                  ▼                    │
│   ┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐          │
│   │  Code Handler   │ │  Data Handler   │ │ General Handler │          │
│   │                 │ │                 │ │                 │          │
│   │ • Code MCP      │ │ • Database MCP  │ │ • Search MCP    │          │
│   │ • GitHub MCP    │ │ • Analytics MCP │ │ • Knowledge MCP │          │
│   └─────────────────┘ └─────────────────┘ └─────────────────┘          │
│                                                                          │
│   USE CASES:                                                            │
│   • Customer support (billing vs technical vs sales)                    │
│   • Development assistant (code vs docs vs testing)                     │
│   • Research (web vs academic vs internal)                              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Router Pattern Implementation

class TaskCategory(Enum):
    CODE = "code"
    DATA = "data"
    RESEARCH = "research"
    GENERAL = "general"


@dataclass
class RouterConfig:
    """Configuration for task routing."""
    category_keywords: Dict[TaskCategory, List[str]] = field(default_factory=dict)
    category_tools: Dict[TaskCategory, List[str]] = field(default_factory=dict)
    
    def __post_init__(self):
        # Default keyword mappings
        if not self.category_keywords:
            self.category_keywords = {
                TaskCategory.CODE: [
                    "code", "function", "bug", "debug", "implement",
                    "python", "javascript", "api", "error", "exception"
                ],
                TaskCategory.DATA: [
                    "data", "database", "query", "sql", "analytics",
                    "chart", "graph", "metrics", "report", "dashboard"
                ],
                TaskCategory.RESEARCH: [
                    "research", "find", "search", "look up", "what is",
                    "explain", "compare", "difference", "how does"
                ]
            }
        
        # Default tool mappings (MCP servers per category)
        if not self.category_tools:
            self.category_tools = {
                TaskCategory.CODE: ["github-mcp", "code-executor-mcp"],
                TaskCategory.DATA: ["database-mcp", "analytics-mcp"],
                TaskCategory.RESEARCH: ["web-search-mcp", "knowledge-mcp"],
                TaskCategory.GENERAL: ["filesystem-mcp"]
            }


class TaskRouter:
    """Routes tasks to appropriate handlers based on classification."""
    
    def __init__(self, config: RouterConfig = None):
        self.config = config or RouterConfig()
    
    def classify(self, task: str) -> TaskCategory:
        """Classify a task into a category."""
        task_lower = task.lower()
        
        # Score each category based on keyword matches
        scores = {}
        for category, keywords in self.config.category_keywords.items():
            score = sum(1 for kw in keywords if kw in task_lower)
            scores[category] = score
        
        # Return highest scoring category, or GENERAL if no matches
        if max(scores.values()) > 0:
            return max(scores, key=scores.get)
        return TaskCategory.GENERAL
    
    def get_tools(self, category: TaskCategory) -> List[str]:
        """Get MCP servers/tools for a category."""
        return self.config.category_tools.get(category, [])
    
    def route(self, task: str) -> Dict[str, Any]:
        """Route a task and return routing decision."""
        category = self.classify(task)
        tools = self.get_tools(category)
        
        return {
            "task": task,
            "category": category.value,
            "recommended_tools": tools,
            "routing_confidence": "high" if len(tools) > 0 else "low"
        }


# Demo
router = TaskRouter()

test_tasks = [
    "Fix the bug in the authentication function",
    "Create a dashboard showing monthly sales data",
    "Research the latest trends in machine learning",
    "Help me organize my files"
]

print("Router Pattern Demo:")
print("=" * 60)
for task in test_tasks:
    result = router.route(task)
    print(f"\nTask: {task}")
    print(f"  → Category: {result['category']}")
    print(f"  → Tools: {result['recommended_tools']}")

In [ ]:
# LLM-Based Router (More Sophisticated)
LLM_ROUTER_PROMPT = '''
You are a task router. Classify the user's request into exactly ONE category:

CATEGORIES:
- CODE: Programming, debugging, code review, implementation
- DATA: Database queries, analytics, visualizations, reports
- RESEARCH: Information lookup, comparisons, explanations
- GENERAL: File management, scheduling, other tasks

Respond with ONLY the category name, nothing else.

User request: {task}

Category:
'''

class LLMRouter:
    """Router that uses LLM for classification."""
    
    def __init__(self, llm_client):
        self.llm = llm_client
        self.tool_mapping = {
            "CODE": ["github-mcp", "code-executor-mcp", "linter-mcp"],
            "DATA": ["database-mcp", "analytics-mcp", "visualization-mcp"],
            "RESEARCH": ["web-search-mcp", "arxiv-mcp", "wikipedia-mcp"],
            "GENERAL": ["filesystem-mcp", "calendar-mcp"]
        }
    
    async def classify(self, task: str) -> str:
        """Use LLM to classify the task."""
        prompt = LLM_ROUTER_PROMPT.format(task=task)
        # response = await self.llm.complete(prompt)
        # return response.strip().upper()
        return "CODE"  # Placeholder
    
    async def route(self, task: str) -> Dict[str, Any]:
        """Route task using LLM classification."""
        category = await self.classify(task)
        tools = self.tool_mapping.get(category, [])
        
        return {
            "task": task,
            "category": category,
            "tools": tools
        }

print("LLM Router Prompt Template:")
print(LLM_ROUTER_PROMPT)

### Router with MCP Integration

In [ ]:
# MCP Server that acts as a Router
ROUTER_MCP_SERVER = '''
from fastmcp import FastMCP
from enum import Enum
from typing import List

mcp = FastMCP("TaskRouter")

# Category definitions with associated MCP servers
ROUTING_TABLE = {
    "code": {
        "keywords": ["code", "bug", "function", "implement", "debug"],
        "servers": ["github", "code-executor"],
        "description": "Programming and development tasks"
    },
    "data": {
        "keywords": ["data", "query", "database", "analytics", "chart"],
        "servers": ["postgres", "analytics"],
        "description": "Data analysis and database tasks"
    },
    "research": {
        "keywords": ["search", "find", "research", "look up", "explain"],
        "servers": ["web-search", "arxiv"],
        "description": "Research and information retrieval"
    }
}


@mcp.tool()
def classify_task(task: str) -> dict:
    """Classify a task and recommend appropriate MCP servers.
    
    Args:
        task: The task description to classify
    
    Returns:
        Classification result with recommended servers
    """
    task_lower = task.lower()
    
    # Score each category
    scores = {}
    for category, config in ROUTING_TABLE.items():
        score = sum(1 for kw in config["keywords"] if kw in task_lower)
        scores[category] = score
    
    # Get best category
    best_category = max(scores, key=scores.get) if max(scores.values()) > 0 else "general"
    
    if best_category in ROUTING_TABLE:
        return {
            "category": best_category,
            "confidence": scores[best_category] / len(ROUTING_TABLE[best_category]["keywords"]),
            "recommended_servers": ROUTING_TABLE[best_category]["servers"],
            "description": ROUTING_TABLE[best_category]["description"]
        }
    
    return {
        "category": "general",
        "confidence": 0.5,
        "recommended_servers": ["filesystem"],
        "description": "General purpose tasks"
    }


@mcp.tool()
def get_routing_table() -> dict:
    """Get the complete routing table configuration.
    
    Returns:
        All categories with their keywords and servers
    """
    return ROUTING_TABLE
'''

print("Router MCP Server:")
print("=" * 60)
print(ROUTER_MCP_SERVER)

## 4. Pattern 2: Parallelization

### Concept

Execute multiple independent tasks simultaneously for efficiency:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      PARALLELIZATION PATTERN                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   TWO VARIANTS:                                                         │
│                                                                          │
│   A. SECTIONING                    B. VOTING                            │
│   ══════════════                   ════════                             │
│   Split task into                  Same task to                         │
│   independent parts                multiple models                      │
│                                                                          │
│        Task                             Task                            │
│          │                                │                             │
│    ┌─────┼─────┐                    ┌─────┼─────┐                       │
│    ▼     ▼     ▼                    ▼     ▼     ▼                       │
│  [P1]  [P2]  [P3]               [M1]   [M2]  [M3]                       │
│    │     │     │                    │     │     │                       │
│    └─────┼─────┘                    └─────┼─────┘                       │
│          ▼                                ▼                             │
│     Aggregate                         Majority                          │
│                                        Vote                             │
│                                                                          │
│   Example:                         Example:                             │
│   Analyze 3 documents              Code review by                       │
│   in parallel                      3 different models                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Parallelization Pattern Implementation
import asyncio
from typing import List, Callable, Any
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

@dataclass
class ParallelTask:
    """Represents a task to be executed in parallel."""
    name: str
    func: Callable
    args: tuple = ()
    kwargs: dict = field(default_factory=dict)


class ParallelExecutor:
    """Execute multiple tasks in parallel and aggregate results."""
    
    def __init__(self, max_workers: int = 5):
        self.max_workers = max_workers
    
    async def execute_all(self, tasks: List[ParallelTask]) -> Dict[str, Any]:
        """Execute all tasks in parallel."""
        results = {}
        
        async def run_task(task: ParallelTask):
            try:
                if asyncio.iscoroutinefunction(task.func):
                    result = await task.func(*task.args, **task.kwargs)
                else:
                    # Run sync functions in thread pool
                    loop = asyncio.get_event_loop()
                    result = await loop.run_in_executor(
                        None, lambda: task.func(*task.args, **task.kwargs)
                    )
                return task.name, {"status": "success", "result": result}
            except Exception as e:
                return task.name, {"status": "error", "error": str(e)}
        
        # Execute all tasks concurrently
        completed = await asyncio.gather(*[run_task(t) for t in tasks])
        
        for name, result in completed:
            results[name] = result
        
        return results
    
    def aggregate(self, results: Dict[str, Any], strategy: str = "all") -> Any:
        """Aggregate results based on strategy."""
        successful = {
            k: v["result"] 
            for k, v in results.items() 
            if v["status"] == "success"
        }
        
        if strategy == "all":
            return successful
        elif strategy == "first":
            return next(iter(successful.values()), None)
        elif strategy == "vote":
            # Return most common result
            from collections import Counter
            counts = Counter(str(v) for v in successful.values())
            return counts.most_common(1)[0][0] if counts else None
        
        return successful


# Demo with simulated MCP tool calls
def analyze_document(doc_id: str) -> dict:
    """Simulated document analysis."""
    import time
    time.sleep(0.1)  # Simulate work
    return {"doc_id": doc_id, "word_count": 1000 + hash(doc_id) % 500}

# Create parallel tasks
tasks = [
    ParallelTask("doc1", analyze_document, ("document_1",)),
    ParallelTask("doc2", analyze_document, ("document_2",)),
    ParallelTask("doc3", analyze_document, ("document_3",)),
]

print("Parallelization Demo:")
print("=" * 60)
print(f"Executing {len(tasks)} tasks in parallel...")

# Note: In a Jupyter notebook, you'd use:
# results = await executor.execute_all(tasks)
print("\nTasks created for parallel execution:")
for task in tasks:
    print(f"  - {task.name}: {task.func.__name__}({task.args})")

## 5. Pattern 3: Orchestrator-Workers

### Concept

A central **orchestrator** breaks down complex tasks and delegates to specialized **workers**:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR-WORKERS PATTERN                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                      ┌─────────────────────┐                            │
│   Complex Task ─────►│    ORCHESTRATOR     │                            │
│                      │                     │                            │
│                      │  1. Analyze task    │                            │
│                      │  2. Break into subs │                            │
│                      │  3. Assign workers  │                            │
│                      │  4. Synthesize      │                            │
│                      └──────────┬──────────┘                            │
│                                 │                                       │
│              ┌──────────────────┼──────────────────┐                    │
│              │                  │                  │                    │
│              ▼                  ▼                  ▼                    │
│   ┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐          │
│   │  WORKER: Code   │ │ WORKER: Research│ │  WORKER: Test   │          │
│   │                 │ │                 │ │                 │          │
│   │ • Write code    │ │ • Find docs     │ │ • Write tests   │          │
│   │ • code-mcp      │ │ • search-mcp    │ │ • pytest-mcp    │          │
│   └────────┬────────┘ └────────┬────────┘ └────────┬────────┘          │
│            │                   │                   │                    │
│            └───────────────────┼───────────────────┘                    │
│                                ▼                                        │
│                      ┌─────────────────────┐                            │
│                      │   ORCHESTRATOR      │                            │
│                      │   Synthesizes       │                            │
│                      │   Final Result      │                            │
│                      └─────────────────────┘                            │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Orchestrator-Workers Pattern Implementation

@dataclass
class SubTask:
    """A subtask created by the orchestrator."""
    id: str
    description: str
    worker_type: str
    dependencies: List[str] = field(default_factory=list)
    status: str = "pending"
    result: Any = None


@dataclass
class Worker:
    """A specialized worker agent."""
    name: str
    capabilities: List[str]
    mcp_servers: List[str]
    
    def can_handle(self, task_type: str) -> bool:
        """Check if worker can handle this task type."""
        return task_type in self.capabilities


class Orchestrator:
    """Central orchestrator that manages workers and tasks."""
    
    def __init__(self):
        self.workers: List[Worker] = []
        self.subtasks: Dict[str, SubTask] = {}
    
    def register_worker(self, worker: Worker):
        """Register a worker."""
        self.workers.append(worker)
    
    def decompose_task(self, task: str) -> List[SubTask]:
        """Break down a complex task into subtasks.
        
        In production, this would use an LLM to analyze the task.
        """
        # Simplified decomposition logic
        subtasks = []
        
        if "build" in task.lower() or "create" in task.lower():
            subtasks = [
                SubTask("1", "Research requirements", "research"),
                SubTask("2", "Design architecture", "design", ["1"]),
                SubTask("3", "Implement code", "code", ["2"]),
                SubTask("4", "Write tests", "testing", ["3"]),
                SubTask("5", "Documentation", "docs", ["3"])
            ]
        else:
            subtasks = [
                SubTask("1", "Analyze request", "research"),
                SubTask("2", "Execute task", "general", ["1"])
            ]
        
        for st in subtasks:
            self.subtasks[st.id] = st
        
        return subtasks
    
    def assign_worker(self, subtask: SubTask) -> Optional[Worker]:
        """Find the best worker for a subtask."""
        for worker in self.workers:
            if worker.can_handle(subtask.worker_type):
                return worker
        return None
    
    def get_execution_order(self) -> List[List[SubTask]]:
        """Get subtasks grouped by execution wave (respecting dependencies)."""
        waves = []
        completed = set()
        
        while len(completed) < len(self.subtasks):
            wave = []
            for task_id, task in self.subtasks.items():
                if task_id in completed:
                    continue
                # Check if all dependencies are complete
                if all(dep in completed for dep in task.dependencies):
                    wave.append(task)
            
            if not wave:
                break  # Circular dependency or error
            
            waves.append(wave)
            completed.update(t.id for t in wave)
        
        return waves


# Demo
orchestrator = Orchestrator()

# Register workers
orchestrator.register_worker(Worker(
    name="ResearchAgent",
    capabilities=["research"],
    mcp_servers=["web-search-mcp", "arxiv-mcp"]
))
orchestrator.register_worker(Worker(
    name="CodeAgent",
    capabilities=["code", "design"],
    mcp_servers=["github-mcp", "code-executor-mcp"]
))
orchestrator.register_worker(Worker(
    name="TestAgent",
    capabilities=["testing"],
    mcp_servers=["pytest-mcp"]
))
orchestrator.register_worker(Worker(
    name="DocsAgent",
    capabilities=["docs", "general"],
    mcp_servers=["markdown-mcp"]
))

# Decompose a complex task
task = "Build a REST API for user authentication"
subtasks = orchestrator.decompose_task(task)

print("Orchestrator-Workers Demo:")
print("=" * 60)
print(f"\nOriginal Task: {task}")
print(f"\nDecomposed into {len(subtasks)} subtasks:")

waves = orchestrator.get_execution_order()
for i, wave in enumerate(waves):
    print(f"\n  Wave {i + 1} (can run in parallel):")
    for st in wave:
        worker = orchestrator.assign_worker(st)
        print(f"    [{st.id}] {st.description}")
        print(f"        → Worker: {worker.name if worker else 'None'}")
        print(f"        → MCP: {worker.mcp_servers if worker else []}")

### Real-World Example: Travel Planning Orchestrator

From the MCP research docs - a travel planning system using orchestrator-workers:

In [ ]:
# Travel Planning Orchestrator Example
TRAVEL_ORCHESTRATOR = '''
┌─────────────────────────────────────────────────────────────────────────┐
│                    TRAVEL PLANNING ORCHESTRATOR                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   User: "Plan a 5-day trip to Tokyo in April"                           │
│                                                                          │
│                      ┌─────────────────────┐                            │
│                      │    ORCHESTRATOR     │                            │
│                      │   (Trip Planner)    │                            │
│                      └──────────┬──────────┘                            │
│                                 │                                       │
│         ┌───────────┬──────────┼──────────┬───────────┐                │
│         ▼           ▼          ▼          ▼           ▼                │
│   ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐    │
│   │ FLIGHTS  │ │  HOTELS  │ │ACTIVITIES│ │  DINING  │ │ WEATHER  │    │
│   │  Worker  │ │  Worker  │ │  Worker  │ │  Worker  │ │  Worker  │    │
│   └────┬─────┘ └────┬─────┘ └────┬─────┘ └────┬─────┘ └────┬─────┘    │
│        │            │            │            │            │           │
│        ▼            ▼            ▼            ▼            ▼           │
│   [flights-mcp] [booking-mcp] [tripadvisor] [yelp-mcp] [weather-mcp]  │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   SYNTHESIS:                                                            │
│   • Orchestrator receives all worker results                            │
│   • Resolves conflicts (hotel near activities)                          │
│   • Creates coherent day-by-day itinerary                               │
│   • Generates final trip plan with bookings                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(TRAVEL_ORCHESTRATOR)

## 6. Pattern 4: Evaluator-Optimizer

### Concept

An **iterative refinement loop** where output is evaluated and improved:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    EVALUATOR-OPTIMIZER PATTERN                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                      ┌─────────────────┐                                │
│   Input ────────────►│    GENERATOR    │                                │
│                      │                 │                                │
│                      │ Create initial  │                                │
│                      │ output          │                                │
│                      └────────┬────────┘                                │
│                               │                                         │
│                               ▼                                         │
│                      ┌─────────────────┐                                │
│               ┌─────►│   EVALUATOR     │                                │
│               │      │                 │                                │
│               │      │ • Check quality │                                │
│               │      │ • Score output  │                                │
│               │      │ • Identify gaps │                                │
│               │      └────────┬────────┘                                │
│               │               │                                         │
│               │               ▼                                         │
│               │      ┌─────────────────┐                                │
│               │      │  PASS/FAIL?     │                                │
│               │      └────────┬────────┘                                │
│               │               │                                         │
│               │    ┌──────────┴──────────┐                              │
│               │    │                     │                              │
│               │   FAIL                  PASS                            │
│               │    │                     │                              │
│               │    ▼                     ▼                              │
│               │ ┌─────────────┐    ┌─────────────┐                      │
│               │ │  OPTIMIZER  │    │   OUTPUT    │                      │
│               │ │             │    │   Final     │                      │
│               │ │ Improve     │    │   Result    │                      │
│               │ │ based on    │    └─────────────┘                      │
│               │ │ feedback    │                                         │
│               │ └──────┬──────┘                                         │
│               │        │                                                │
│               └────────┘                                                │
│                                                                          │
│   MAX ITERATIONS: Prevent infinite loops (typically 3-5)                │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Evaluator-Optimizer Pattern Implementation

@dataclass
class EvaluationResult:
    """Result of evaluating generated content."""
    passed: bool
    score: float  # 0.0 to 1.0
    feedback: List[str]
    criteria_scores: Dict[str, float] = field(default_factory=dict)


class EvaluatorOptimizer:
    """Iterative refinement loop for content generation."""
    
    def __init__(
        self,
        generator: Callable,
        evaluator: Callable,
        optimizer: Callable,
        max_iterations: int = 3,
        pass_threshold: float = 0.8
    ):
        self.generator = generator
        self.evaluator = evaluator
        self.optimizer = optimizer
        self.max_iterations = max_iterations
        self.pass_threshold = pass_threshold
    
    def run(self, input_data: Any) -> Dict[str, Any]:
        """Run the evaluation-optimization loop."""
        history = []
        current_output = None
        
        for iteration in range(self.max_iterations):
            # Generate (or optimize)
            if current_output is None:
                current_output = self.generator(input_data)
            else:
                current_output = self.optimizer(
                    current_output,
                    history[-1]["evaluation"].feedback
                )
            
            # Evaluate
            evaluation = self.evaluator(current_output)
            
            history.append({
                "iteration": iteration + 1,
                "output": current_output,
                "evaluation": evaluation
            })
            
            # Check if passed
            if evaluation.passed or evaluation.score >= self.pass_threshold:
                return {
                    "success": True,
                    "iterations": iteration + 1,
                    "final_output": current_output,
                    "final_score": evaluation.score,
                    "history": history
                }
        
        # Max iterations reached
        return {
            "success": False,
            "iterations": self.max_iterations,
            "final_output": current_output,
            "final_score": history[-1]["evaluation"].score,
            "history": history
        }


# Example: Code Review Loop
def generate_code(task: str) -> str:
    """Generate initial code (simulated)."""
    return f"def solve():\n    # TODO: {task}\n    pass"

def evaluate_code(code: str) -> EvaluationResult:
    """Evaluate code quality (simulated)."""
    issues = []
    score = 0.5
    
    if "TODO" in code:
        issues.append("Contains TODO comments")
    else:
        score += 0.2
    
    if "pass" in code and code.count("\n") < 5:
        issues.append("Implementation is incomplete")
    else:
        score += 0.2
    
    if "def " in code:
        score += 0.1
    
    return EvaluationResult(
        passed=score >= 0.8,
        score=score,
        feedback=issues if issues else ["Code looks good!"]
    )

def optimize_code(code: str, feedback: List[str]) -> str:
    """Improve code based on feedback (simulated)."""
    improved = code
    
    if "TODO" in improved:
        improved = improved.replace("# TODO:", "# Implementation for:")
        improved = improved.replace("pass", "return 'implemented'")
    
    return improved


# Demo
loop = EvaluatorOptimizer(
    generator=generate_code,
    evaluator=evaluate_code,
    optimizer=optimize_code,
    max_iterations=3,
    pass_threshold=0.8
)

result = loop.run("calculate fibonacci")

print("Evaluator-Optimizer Demo:")
print("=" * 60)
print(f"\nSuccess: {result['success']}")
print(f"Iterations: {result['iterations']}")
print(f"Final Score: {result['final_score']}")
print(f"\nFinal Output:")
print(result['final_output'])
print(f"\nHistory:")
for h in result['history']:
    print(f"  Iteration {h['iteration']}: score={h['evaluation'].score:.2f}")

### MCP-Powered Code Review Loop

In [ ]:
# Code Review with MCP Tools
CODE_REVIEW_LOOP = '''
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP-POWERED CODE REVIEW LOOP                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   GENERATE                     EVALUATE                 OPTIMIZE        │
│   ════════                     ════════                 ════════        │
│                                                                          │
│   ┌─────────────┐         ┌─────────────┐         ┌─────────────┐      │
│   │   claude    │         │  linter-mcp │         │   claude    │      │
│   │   (write)   │────────►│  pytest-mcp │────────►│  (improve)  │      │
│   │             │         │security-mcp │         │             │      │
│   └─────────────┘         └─────────────┘         └──────┬──────┘      │
│                                                          │             │
│                                                          │             │
│   MCP TOOLS USED:                                        ▼             │
│   ═══════════════                                   Loop back          │
│                                                     if needed          │
│   GENERATE:                                                            │
│   • code-executor/write_file                                           │
│                                                                         │
│   EVALUATE:                                                            │
│   • linter/check_style (pylint, flake8)                                │
│   • linter/check_types (mypy)                                          │
│   • pytest/run_tests                                                   │
│   • security/scan_vulnerabilities (bandit)                             │
│                                                                         │
│   OPTIMIZE:                                                            │
│   • LLM with feedback context                                          │
│   • code-executor/write_file (updated version)                         │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(CODE_REVIEW_LOOP)

## 7. Pattern 5: Autonomous Agent

### Concept

A **self-directed agent** that plans, acts, and adapts:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    AUTONOMOUS AGENT PATTERN                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                     ┌──────────────────────────┐                        │
│                     │      AGENT CORE          │                        │
│                     │                          │                        │
│                     │  ┌──────────────────┐   │                        │
│   Goal ────────────►│  │     PLANNER      │   │                        │
│                     │  │  Create strategy │   │                        │
│                     │  └────────┬─────────┘   │                        │
│                     │           │             │                        │
│                     │           ▼             │                        │
│                     │  ┌──────────────────┐   │                        │
│                     │  │     EXECUTOR     │   │◄──── MCP TOOLS        │
│                     │  │  Take actions    │   │      • filesystem     │
│                     │  └────────┬─────────┘   │      • database       │
│                     │           │             │      • web-search     │
│                     │           ▼             │      • code-exec      │
│                     │  ┌──────────────────┐   │                        │
│                     │  │    OBSERVER      │   │                        │
│                     │  │  Check results   │   │                        │
│                     │  └────────┬─────────┘   │                        │
│                     │           │             │                        │
│                     │           ▼             │                        │
│                     │  ┌──────────────────┐   │                        │
│                     │  │    REFLECTOR     │   │◄──── MEMORY           │
│                     │  │  Learn & adapt   │   │      • Past actions   │
│                     │  └────────┬─────────┘   │      • Outcomes       │
│                     │           │             │      • Learnings      │
│                     └───────────┼─────────────┘                        │
│                                 │                                       │
│                         ┌───────┴───────┐                               │
│                         │               │                               │
│                      Goal Met?       Continue                           │
│                         │               │                               │
│                         ▼               └──────► Back to PLANNER        │
│                      OUTPUT                                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Autonomous Agent Implementation

@dataclass
class AgentMemory:
    """Agent's memory of past actions and outcomes."""
    actions: List[Dict[str, Any]] = field(default_factory=list)
    observations: List[str] = field(default_factory=list)
    learnings: List[str] = field(default_factory=list)
    
    def add_action(self, action: str, result: Any, success: bool):
        self.actions.append({
            "action": action,
            "result": result,
            "success": success,
            "timestamp": datetime.now().isoformat()
        })
    
    def add_observation(self, observation: str):
        self.observations.append(observation)
    
    def add_learning(self, learning: str):
        self.learnings.append(learning)
    
    def get_context(self, last_n: int = 5) -> str:
        """Get recent context for the agent."""
        recent_actions = self.actions[-last_n:]
        context = "Recent actions:\n"
        for a in recent_actions:
            status = "✓" if a["success"] else "✗"
            context += f"  {status} {a['action']}\n"
        return context


@dataclass
class AgentPlan:
    """A plan created by the agent."""
    goal: str
    steps: List[str]
    current_step: int = 0
    
    def next_step(self) -> Optional[str]:
        if self.current_step < len(self.steps):
            step = self.steps[self.current_step]
            self.current_step += 1
            return step
        return None
    
    def is_complete(self) -> bool:
        return self.current_step >= len(self.steps)


class AutonomousAgent:
    """Self-directed agent with planning and memory."""
    
    def __init__(self, name: str, mcp_tools: List[str]):
        self.name = name
        self.mcp_tools = mcp_tools
        self.memory = AgentMemory()
        self.current_plan: Optional[AgentPlan] = None
        self.max_iterations = 10
    
    def plan(self, goal: str) -> AgentPlan:
        """Create a plan to achieve the goal.
        
        In production, this would use an LLM with the goal and memory context.
        """
        # Simplified planning logic
        steps = [
            f"1. Analyze the goal: {goal}",
            "2. Gather required information",
            "3. Execute main task",
            "4. Verify results",
            "5. Report completion"
        ]
        
        self.current_plan = AgentPlan(goal=goal, steps=steps)
        return self.current_plan
    
    def execute_step(self, step: str) -> Dict[str, Any]:
        """Execute a single step.
        
        In production, this would call MCP tools based on the step.
        """
        # Simulated execution
        result = {
            "step": step,
            "success": True,
            "output": f"Completed: {step}"
        }
        
        self.memory.add_action(step, result["output"], result["success"])
        return result
    
    def observe(self, result: Dict[str, Any]) -> str:
        """Observe and interpret the result."""
        observation = f"Step result: {result['output']}"
        self.memory.add_observation(observation)
        return observation
    
    def reflect(self) -> Optional[str]:
        """Reflect on progress and adapt if needed."""
        # Check if we need to adapt the plan
        failed_actions = [a for a in self.memory.actions if not a["success"]]
        
        if len(failed_actions) > 2:
            learning = "Multiple failures detected - may need to revise approach"
            self.memory.add_learning(learning)
            return learning
        
        return None
    
    def run(self, goal: str) -> Dict[str, Any]:
        """Run the agent loop until goal is achieved or max iterations."""
        self.plan(goal)
        iterations = 0
        
        while not self.current_plan.is_complete() and iterations < self.max_iterations:
            iterations += 1
            
            # Get next step
            step = self.current_plan.next_step()
            if not step:
                break
            
            # Execute
            result = self.execute_step(step)
            
            # Observe
            self.observe(result)
            
            # Reflect
            self.reflect()
        
        return {
            "goal": goal,
            "completed": self.current_plan.is_complete(),
            "iterations": iterations,
            "memory": {
                "actions": len(self.memory.actions),
                "observations": len(self.memory.observations),
                "learnings": self.memory.learnings
            }
        }


# Demo
agent = AutonomousAgent(
    name="ResearchAgent",
    mcp_tools=["web-search-mcp", "filesystem-mcp", "note-taking-mcp"]
)

result = agent.run("Research and summarize recent AI safety papers")

print("Autonomous Agent Demo:")
print("=" * 60)
print(f"\nGoal: {result['goal']}")
print(f"Completed: {result['completed']}")
print(f"Iterations: {result['iterations']}")
print(f"\nMemory Stats:")
print(f"  Actions recorded: {result['memory']['actions']}")
print(f"  Observations: {result['memory']['observations']}")
print(f"\nAgent Context:")
print(agent.memory.get_context())

## 8. Combining Patterns: Multi-Agent Systems

Real-world systems often combine multiple patterns:

In [ ]:
# Combined Pattern Example
COMBINED_ARCHITECTURE = '''
┌─────────────────────────────────────────────────────────────────────────┐
│            COMBINED PATTERNS: ENTERPRISE AI ASSISTANT                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   User Request                                                          │
│        │                                                                │
│        ▼                                                                │
│   ┌─────────────┐                                                       │
│   │   ROUTER    │◄──────────────────── Pattern 1                        │
│   │ (classify)  │                                                       │
│   └──────┬──────┘                                                       │
│          │                                                              │
│    ┌─────┼─────┬─────────────┐                                          │
│    │     │     │             │                                          │
│    ▼     ▼     ▼             ▼                                          │
│ ┌─────┐┌─────┐┌─────┐   ┌─────────┐                                    │
│ │Code ││Data ││ Doc │   │ Complex │                                    │
│ │Path ││Path ││Path │   │  Task   │                                    │
│ └──┬──┘└──┬──┘└──┬──┘   └────┬────┘                                    │
│    │      │      │           │                                          │
│    │      │      │           ▼                                          │
│    │      │      │   ┌──────────────┐                                   │
│    │      │      │   │ ORCHESTRATOR │◄───── Pattern 3                   │
│    │      │      │   └───────┬──────┘                                   │
│    │      │      │           │                                          │
│    │      │      │    ┌──────┼──────┐                                   │
│    │      │      │    ▼      ▼      ▼                                   │
│    │      │      │  [W1]   [W2]   [W3] ◄──── Pattern 2 (parallel)      │
│    │      │      │    │      │      │                                   │
│    │      │      │    └──────┼──────┘                                   │
│    │      │      │           │                                          │
│    │      │      │           ▼                                          │
│    │      │      │   ┌──────────────┐                                   │
│    │      │      │   │   EVALUATE   │◄───── Pattern 4                   │
│    │      │      │   │   OPTIMIZE   │                                   │
│    │      │      │   └───────┬──────┘                                   │
│    │      │      │           │                                          │
│    └──────┴──────┴───────────┘                                          │
│                  │                                                      │
│                  ▼                                                      │
│           Final Response                                                │
│                                                                          │
│   MCP SERVERS USED:                                                     │
│   • Router: classifier-mcp                                              │
│   • Code: github-mcp, code-exec-mcp, linter-mcp                         │
│   • Data: postgres-mcp, analytics-mcp                                   │
│   • Docs: confluence-mcp, search-mcp                                    │
│   • Workers: various specialized MCPs                                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(COMBINED_ARCHITECTURE)

## 9. Best Practices for Agentic MCP Integration

```
┌─────────────────────────────────────────────────────────────────────────┐
│              AGENTIC MCP INTEGRATION BEST PRACTICES                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   1. TOOL SELECTION                                                     │
│   ══════════════════                                                    │
│   • Expose only necessary tools to each agent                           │
│   • Use clear, descriptive tool names                                   │
│   • Provide rich descriptions for LLM understanding                     │
│   • Group related tools in single MCP servers                           │
│                                                                          │
│   2. ERROR HANDLING                                                     │
│   ═════════════════                                                     │
│   • Agents should handle tool failures gracefully                       │
│   • Implement retry logic with backoff                                  │
│   • Provide fallback tools when primary fails                           │
│   • Log all tool interactions for debugging                             │
│                                                                          │
│   3. STATE MANAGEMENT                                                   │
│   ══════════════════                                                    │
│   • Use MCP resources for shared state                                  │
│   • Implement session management for long tasks                         │
│   • Checkpoint progress for resumability                                │
│   • Clean up resources after task completion                            │
│                                                                          │
│   4. SECURITY                                                           │
│   ══════════                                                            │
│   • Validate all tool inputs                                            │
│   • Implement rate limiting per agent                                   │
│   • Use least-privilege tool access                                     │
│   • Audit all sensitive operations                                      │
│                                                                          │
│   5. OBSERVABILITY                                                      │
│   ══════════════                                                        │
│   • Trace agent decisions and tool calls                                │
│   • Monitor latency and success rates                                   │
│   • Alert on unusual patterns                                           │
│   • Provide debugging interfaces                                        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 10. Key Takeaways

### Pattern Selection Guide

| Pattern | Use When | MCP Role |
|---------|----------|----------|
| **Router** | Different task types need different handling | Route to specialized MCP servers |
| **Parallelization** | Independent subtasks can run simultaneously | Execute multiple MCP calls concurrently |
| **Orchestrator-Workers** | Complex tasks need decomposition | Coordinate multiple MCP servers |
| **Evaluator-Optimizer** | Output quality needs iterative improvement | Evaluation tools + generation |
| **Autonomous Agent** | Open-ended goals with uncertainty | Full MCP toolkit with memory |

### Key Principles

1. **Start Simple**: Begin with Router, add complexity as needed
2. **Limit Autonomy**: More autonomy = more risk; add guardrails
3. **Observe Everything**: You can't improve what you can't measure
4. **Design for Failure**: Agents will make mistakes; plan for recovery
5. **Human in the Loop**: Critical decisions should involve humans

### Next Notebook

In **Notebook 9: MCP Gateway Fundamentals**, we'll explore:
- Gateway architecture and benefits
- Building a basic MCP gateway
- Request routing and load balancing
- Authentication at the gateway level

## 11. Exercises

### Exercise 1: Design a Router

Create a router for a customer support system with these categories:
- Billing (refunds, payments, invoices)
- Technical (bugs, errors, how-to)
- Account (login, settings, profile)
- Sales (pricing, features, upgrades)

In [ ]:
# Exercise 1: Design your customer support router

class SupportRouter:
    """Router for customer support requests."""
    
    def __init__(self):
        # Define your routing table
        self.routing_table = {
            # Your categories here
        }
    
    def classify(self, ticket: str) -> str:
        """Classify a support ticket."""
        # Your implementation here
        pass
    
    def get_mcp_servers(self, category: str) -> List[str]:
        """Get appropriate MCP servers for the category."""
        # Your implementation here
        pass

# Test your router
router = SupportRouter()
test_tickets = [
    "I was charged twice for my subscription",
    "The app crashes when I try to upload files",
    "How do I change my password?",
    "What's included in the Pro plan?"
]

print("Test your support router:")
for ticket in test_tickets:
    print(f"\nTicket: {ticket}")
    # print(f"Category: {router.classify(ticket)}")

### Exercise 2: Implement an Evaluator

Create an evaluator for generated documentation that checks:
- Completeness (all sections present)
- Clarity (readability score)
- Accuracy (code examples work)

In [ ]:
# Exercise 2: Create a documentation evaluator

class DocEvaluator:
    """Evaluates generated documentation quality."""
    
    REQUIRED_SECTIONS = ["Overview", "Installation", "Usage", "API Reference"]
    
    def evaluate(self, doc: str) -> EvaluationResult:
        """Evaluate documentation quality."""
        feedback = []
        scores = {}
        
        # Check completeness
        # Your implementation here
        
        # Check clarity
        # Your implementation here
        
        # Calculate overall score
        overall_score = 0.0  # Your calculation
        
        return EvaluationResult(
            passed=overall_score >= 0.8,
            score=overall_score,
            feedback=feedback,
            criteria_scores=scores
        )

# Test your evaluator
test_doc = """
# My Library

## Overview
A simple library for doing things.

## Installation
pip install mylibrary
"""

evaluator = DocEvaluator()
# result = evaluator.evaluate(test_doc)
# print(f"Score: {result.score}")
# print(f"Feedback: {result.feedback}")

### Exercise 3: Design an Orchestrator

Design an orchestrator for a "Write a Blog Post" task that coordinates:
- Research worker (web search)
- Outline worker (structure)
- Writer worker (content)
- Editor worker (polish)

In [ ]:
# Exercise 3: Design your blog post orchestrator

class BlogPostOrchestrator:
    """Orchestrates blog post creation."""
    
    def __init__(self):
        self.workers = {}
        # Register your workers
    
    def decompose(self, topic: str) -> List[SubTask]:
        """Break down blog post creation into subtasks."""
        # Your implementation
        pass
    
    def execute(self, topic: str) -> str:
        """Execute the full blog post creation workflow."""
        # Your implementation
        pass

# Test your orchestrator design
orchestrator = BlogPostOrchestrator()
# subtasks = orchestrator.decompose("The Future of AI Agents")
# for st in subtasks:
#     print(f"{st.id}: {st.description} (depends on: {st.dependencies})")

---

## References

- [Anthropic: Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
- [LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/)
- [CrewAI Multi-Agent Framework](https://www.crewai.io/)
- [AutoGen Multi-Agent Conversations](https://microsoft.github.io/autogen/)

---

**Next Notebook:** [09_MCP_Gateway_Fundamentals.ipynb](./09_MCP_Gateway_Fundamentals.ipynb)